<a href="https://colab.research.google.com/github/SanjaghRecommand/Recommend/blob/Foad/tensorflow_recommenders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow_recommenders

In [ ]:
from typing import Dict, Text
 
import numpy as np
import pandas as pd
import tensorflow as tf
 
import tensorflow_recommenders as tfrs 

In [ ]:
data = pd.read_csv ('/content/drive/MyDrive/Colab Notebooks/sanjaghshare/data/dfn3share.csv')
# dffull = pd.read_csv ('/content/drive/MyDrive/Colab Notebooks/sanjaghshare/data/taskfullshare.csv')

In [ ]:
data

,Unnamed: 0,user,service,count,subcat,gender
0,0,3646,92,127,15,1
1,1,3646,94,18,15,1
2,2,3646,93,19,15,1
3,3,3646,95,2,17,1
4,4,3646,114,1,15,1
...,...,...,...,...,...,...
1143745,1143745,1476035,525,2,72,1
1143746,1143746,1396700,525,2,72,3
1143747,1143747,1444961,185,2,25,2
1143748,1143748,1507320,207,1,25,2


In [ ]:
user_ids = data["user"].unique()
gender_vocab = tf.keras.layers.experimental.preprocessing.IntegerLookup(vocabulary=data["gender"].unique(), mask_value=None)
user_features = {    "gender": gender_vocab(tf.constant(data["gender"].values))}

In [ ]:
service_ids = data["service"].unique()
category_vocab = tf.keras.layers.experimental.preprocessing.IntegerLookup(vocabulary=data["subcat"].unique(), mask_token=None)
service_features = {  "category": category_vocab(tf.constant(data["subcat"].values)) }

In [ ]:
interactions = tf.data.Dataset.from_tensor_slices((
    tf.constant(data["user"].values),
    tf.constant(data["service"].values),
    tf.constant(data["count"].values)))

In [ ]:
interactions

<_TensorSliceDataset element_spec=(TensorSpec(shape=(), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [ ]:
train_size = int(len(data) * 0.8)
train_data = interactions.take(train_size)
test_data = interactions.skip(train_size)

In [ ]:
class ServiceUsageModel(tfrs.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32
    
    self.user_embeddings = tf.keras.Sequential([
      gender_vocab,
      tf.keras.layers.Embedding(gender_vocab.vocab_size(), embedding_dimension)
    ])
    self.service_embeddings = tf.keras.Sequential([
      category_vocab,
      tf.keras.layers.Embedding(category_vocab.vocab_size(), embedding_dimension)
    ])
    
    self.task = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )
  def compute_loss(self, inputs, targets, training=False):
        user_embeddings = self.user_model(inputs["user"])
        service_embeddings = self.service_model(inputs["service"])

        # Concatenate user and service embeddings with gender and category
        embeddings = tf.concat([user_embeddings, inputs["gender"], inputs["category"], service_embeddings], axis=1)

        # Compute predictions and loss
        predictions = self.task(embeddings, service_embeddings)
        loss = self.task.loss(targets, predictions)

        return loss

  def call(self, features):
    user_embeddings = self.user_embeddings(features["gender"])
    service_embeddings = self.service_embeddings(features["category"])
    return user_embeddings, service_embeddings

model = ServiceUsageModel()

In [ ]:
# model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1), loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.RootMeanSquaredError()])
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

cached_train_data = train_data.shuffle(len(data)).batch(2048).cache()
cached_test_data = test_data.batch(2048).cache()

In [ ]:
model.fit(cached_train_data, epochs=10)


In [ ]:
input_data = {
    "user_id": tf.constant(data["user"].values),
    "service_id": tf.constant(data["service"].values),
    "gender": tf.constant(data["gender"].values),
    "category": tf.constant(data["subcat"].values)
}
target_data = tf.constant(data["count"].values)

# model.fit((input_data, target_data), epochs=10)

In [ ]:
class ServiceUsageModel(tfrs.Model):
    def __init__(self, user_model, service_model):
        super().__init__()
        self.user_model = user_model
        self.service_model = service_model
        self.task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )

    def compute_loss(self, inputs, targets, training=False):
        user_embeddings = self.user_model(inputs["user_id"])
        service_embeddings = self.service_model(inputs["service_id"])

        # Concatenate user and service embeddings with gender and category
        embeddings = tf.concat([user_embeddings, inputs["gender"], inputs["category"], service_embeddings], axis=1)

        # Compute predictions and loss
        predictions = self.task(embeddings, service_embeddings)
        loss = self.task.loss(targets, predictions)

        return loss
        

In [ ]:


EMBEDDING_SIZE = 25
num_users = 1514100
num_services = 589
num_cats = 81
num_genders=3
embedding_dim=50

In [ ]:
user_id = tf.keras.Input(shape=(), dtype=tf.string, name="user_id")
gender = tf.keras.Input(shape=(), dtype=tf.string, name="gender")
service_id = tf.keras.Input(shape=(), dtype=tf.string, name="service_id")

count = tf.keras.Input(shape=(), dtype=tf.float32, name="count")
user_id_embedding = tf.keras.layers.Embedding(input_dim=num_users, output_dim=embedding_dim, name="user_id_embedding")
gender_embedding = tf.keras.layers.Embedding(input_dim=num_genders, output_dim=embedding_dim, name="gender_embedding")
service_id_embedding = tf.keras.layers.Embedding(input_dim=num_services, output_dim=embedding_dim, name="service_id_embedding")

In [ ]:
user_id_embed = user_id_embedding(user_id)
gender_embed = gender_embedding(gender)
service_id_embed = service_id_embedding(service_id)

In [ ]:
concat_layer = tf.keras.layers.Concatenate()([user_id_embed, gender_embed, service_id_embed])
dense_layer = tf.keras.layers.Dense(units=64, activation="relu")(concat_layer)
count_pred = tf.keras.layers.Dense(units=1, activation="linear")(dense_layer)

In [ ]:
model = tf.keras.Model(inputs=[user_id, gender, service_id, count], outputs=count_pred)


In [ ]:
def compute_loss(labels, predictions):
    mse = tf.keras.losses.MeanSquaredError()
    loss = mse(labels, predictions)
    return loss

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1), loss=compute_loss)


In [ ]:
model.fit( tf.constant(data["user"].values), tf.constant(data["gender"].values),
          tf.constant(data["service"].values),  tf.constant(data["count"].values),
     tf.constant(data["count"].values))

# input_data = {
#     "user_id": tf.constant(data["user"].values),
#     "service_id": tf.constant(data["service"].values),
#     "gender": tf.constant(data["gender"].values),
#     "category": tf.constant(data["subcat"].values)
# }
# target_data = tf.constant(data["count"].values)

ValueError: ignored